<a href="https://colab.research.google.com/github/fjadidi2001/DataScienceJourney/blob/master/Face_March2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics roboflow opencv-python pillow numpy


In [5]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from roboflow import Roboflow
from ultralytics import YOLO

# Connect to Roboflow and download dataset
rf = Roboflow(api_key="2IGtFaicFMGaMwb2mX8A")
project = rf.workspace("mohamed-traore-2ekkp").project("face-detection-mik1i")
version = project.version(25)

# Download the dataset in YOLOv8 format
version.download("yolov8")

# Set paths
dataset_path = "./face-detection-mik1i-25"
train_yaml = os.path.join(dataset_path, "data.yaml")

# Train YOLOv8 model
def train_yolo_model(yaml_path, epochs=50, imgsz=640):
    # Initialize YOLOv8 model (can be 'yolov8n.pt', 'yolov8s.pt', 'yolov8m.pt', 'yolov8l.pt', 'yolov8x.pt')
    model = YOLO('yolov8n.pt')  # Start with a pre-trained model

    # Train the model
    results = model.train(
        data=yaml_path,
        epochs=epochs,
        imgsz=imgsz,
        patience=10,  # Early stopping patience
        batch=16,
        device=0 if torch.cuda.is_available() else 'cpu',
        name="face_detection_model"
    )

    return model

def detect_faces(model, image_path, conf_threshold=0.25):
    # Load image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Error: Could not load image at {image_path}")
        return None

    # Run inference
    results = model(img, conf=conf_threshold)

    # Get detections
    detections = results[0]

    # Convert BGR to RGB for matplotlib
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Draw bounding boxes
    for detection in detections.boxes.data.tolist():
        x1, y1, x2, y2, confidence, class_id = detection

        # Only process if confidence is above threshold
        if confidence >= conf_threshold:
            # Draw bounding box
            cv2.rectangle(img_rgb,
                         (int(x1), int(y1)),
                         (int(x2), int(y2)),
                         (0, 255, 0), 2)

            # Add label
            label = f"Face: {confidence:.2f}"
            cv2.putText(img_rgb, label, (int(x1), int(y1) - 10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return img_rgb, detections

# Example usage
if __name__ == "__main__":
    import torch

    # Train the model (or load a pre-trained one)
    try:
        # Try to load existing model first
        model_path = './runs/detect/face_detection_model/weights/best.pt'
        if os.path.exists(model_path):
            model = YOLO(model_path)
            print("Loaded pre-trained model from", model_path)
        else:
            print("Training new model...")
            model = train_yolo_model(train_yaml, epochs=30)
            print("Model training complete")
    except Exception as e:
        print(f"Error in model training/loading: {e}")
        exit(1)

    # Define a function to process a directory of images
    def process_directory(directory_path, output_dir='./output_images'):
        os.makedirs(output_dir, exist_ok=True)

        # Get all image files
        valid_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
        image_files = [f for f in os.listdir(directory_path)
                      if os.path.isfile(os.path.join(directory_path, f)) and
                      os.path.splitext(f)[1].lower() in valid_extensions]

        print(f"Found {len(image_files)} images to process")

        for img_file in image_files:
            img_path = os.path.join(directory_path, img_file)
            try:
                print(f"Processing {img_file}...")
                img_rgb, detections = detect_faces(model, img_path)

                if img_rgb is not None:
                    # Save the output image
                    output_path = os.path.join(output_dir, f"detected_{img_file}")
                    plt.figure(figsize=(10, 8))
                    plt.imshow(img_rgb)
                    plt.axis('off')
                    plt.savefig(output_path)
                    plt.close()

                    # Count detections
                    boxes = detections.boxes
                    print(f"  Detected {len(boxes)} faces in {img_file}")
            except Exception as e:
                print(f"  Error processing {img_file}: {e}")

    # Example: Process test directory
    test_dir = os.path.join(dataset_path, "test", "images")
    process_directory(test_dir)

    # You can also process a single image
    def process_single_image(image_path):
        img_rgb, detections = detect_faces(model, image_path)
        if img_rgb is not None:
            plt.figure(figsize=(12, 10))
            plt.imshow(img_rgb)
            plt.axis('off')
            plt.title(f"Face Detection - Found {len(detections.boxes)} faces")
            plt.show()

    # Uncomment to process a specific image
    # process_single_image("path/to/your/image.jpg")

loading Roboflow workspace...
loading Roboflow project...
Training new model...
Ultralytics 8.3.81 🚀 Python-3.11.11 torch-2.5.1+cu124 CPU (AMD EPYC 7B12)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=./face-detection-mik1i-25/data.yaml, epochs=30, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=face_detection_model4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False,

FileNotFoundError: [Errno 2] No such file or directory: './face-detection-mik1i-25/test/images'